# Fouille données

In [2]:
!pip install pandas numpy matplotlib seaborn folium scikit-learn

     -------------------------------------- 113.4/113.4 kB 3.3 MB/s eta 0:00:00
     ---------------------------------------- 93.9/93.9 kB 5.2 MB/s eta 0:00:00
     -------------------------------------- 134.9/134.9 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.11.1 requires ruamel-yaml, which is not installed.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from collections import Counter

%matplotlib inline
sns.set(style="whitegrid")

In [5]:
# Chargement du fichier
# on_bad_lines='skip' permet de sauter les lignes mal formées sans planter
# On enlève engine='python' car le moteur par défaut (C) est souvent plus tolérant

df = pd.read_csv('flickr_data2.csv', on_bad_lines='skip')

print("Données brutes :")
display(df.head())

Données brutes :


C:\Users\jerom\AppData\Local\Temp\ipykernel_25956\831863605.py:5: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('flickr_data2.csv', on_bad_lines='skip')


,id,user,lat,long,tags,title,date_taken_minute,date_taken_hour,date_taken_day,date_taken_month,date_taken_year,date_upload_minute,date_upload_hour,date_upload_day,date_upload_month,date_upload_year,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,4395181099,30624617@N03,45.754858,4.821710,"chair,lyon,rhône,chaise,rhônealpes",Chaises avec vue,11.0,15,28,2,2010,23,20,28.0,2,2010.0,NaN,NaN,NaN
1,4394748717,35853470@N00,45.753270,4.862953,NaN,NaN,51.0,17,28,2,2010,52,17,28.0,2,2010.0,NaN,NaN,NaN
2,4394694699,11817998@N05,45.760655,4.846564,"365,iphone",59/365 - R46 V103 B163,29.0,17,28,2,2010,33,17,28.0,2,2010.0,NaN,NaN,NaN
3,4394803790,11545749@N06,45.784000,4.874072,"nin,nineinchnails,gift,screening,toiou,avott",2010-01-29 Toiou Avott Lyon,15.0,20,28,1,2010,38,12,28.0,2,2010.0,NaN,NaN,NaN
4,4394803554,11545749@N06,45.784000,4.874072,"lyon,nin,nineinchnails,gift,screening,toiou,avott",2010-01-28 Toiou Avott Lyon,10.0,20,28,1,2010,38,12,28.0,2,2010.0,NaN,NaN,NaN


In [6]:
print("Colonnes du DataFrame :")
print(df.columns.tolist())
print("\nInfo du DataFrame :")
df.info()

Colonnes du DataFrame :
['id', ' user', ' lat', ' long', ' tags', ' title', ' date_taken_minute', ' date_taken_hour', ' date_taken_day', ' date_taken_month', ' date_taken_year', ' date_upload_minute', ' date_upload_hour', ' date_upload_day', ' date_upload_month', ' date_upload_year', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18']

Info du DataFrame :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420240 entries, 0 to 420239
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   420240 non-null  int64  
 1    user                420240 non-null  object 
 2    lat                 420240 non-null  float64
 3    long                420240 non-null  float64
 4    tags                316730 non-null  object 
 5    title               381911 non-null  object 
 6    date_taken_minute   420239 non-null  float64
 7    date_taken_hour     420240 non-null  int64  
 8    date_taken_day      42

In [ ]:
# Histogramme de la colonne année (date_taken_year)
plt.figure(figsize=(10, 6))
sns.histplot(df[' date_taken_year'], bins=20, kde=True)
plt.title('Histogramme des années de prise des photos')
plt.xlabel('Année')
plt.ylabel('Nombre de photos')
plt.show()